In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import time
import pandas as pd
import re
import random

In [2]:
url = 'https://www.linkedin.com/jobs/search/?currentJobId=3484423747&f_E=2&f_JT=F&f_T=25190&f_TPR=r2592000&geoId=103644278&keywords=Data%20Scientist&location=United%20States&refresh=true&sortBy=DD'

In [3]:
wd = webdriver.Chrome(executable_path='/Users/yinirong/Downloads/chromedriver_mac64/chromedriver')
# Headless browser
# opts = webdriver.ChromeOptions()
# opts.add_argument("--headless=new")
# wd = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opts)
wd.get(url)

#no_of_jobs = 1
no_of_jobs = int(re.sub("[^0-9]", "", 
                        wd.find_element(By.CSS_SELECTOR, 'h1>span').get_attribute('innerText')))

start = time.time()
i = 2
while i <= int(no_of_jobs/25)+1: 
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    i = i + 1
    try:
        wd.find_element(By.XPATH, '/html/body/div[1]/div/main/section[2]/button').click()
        time.sleep(5)
    except Exception as e:
        print(e)
        pass
        time.sleep(5)
end = time.time()
print(f'Run time to load all jobs: {(end - start)/60} minutes')

/var/folders/ty/slr4xft54418twt2ykv4hn1h0000gn/T/ipykernel_510/18712323.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(executable_path='/Users/yinirong/Downloads/chromedriver_mac64/chromedriver')


Message: element not interactable
  (Session info: chrome=111.0.5563.146)
Stacktrace:
0   chromedriver                        0x00000001076cc428 chromedriver + 4899880
1   chromedriver                        0x0000000107649a23 chromedriver + 4364835
2   chromedriver                        0x0000000107293a71 chromedriver + 473713
3   chromedriver                        0x00000001072d8d78 chromedriver + 757112
4   chromedriver                        0x00000001072c9cf3 chromedriver + 695539
5   chromedriver                        0x00000001072fd562 chromedriver + 906594
6   chromedriver                        0x00000001072c9361 chromedriver + 693089
7   chromedriver                        0x00000001072fd71e chromedriver + 907038
8   chromedriver                        0x0000000107318b5b chromedriver + 1018715
9   chromedriver                        0x00000001072fd333 chromedriver + 906035
10  chromedriver                        0x00000001072c755f chromedriver + 685407
11  chromedriver    

In [4]:
job_lists = wd.find_element(By.CLASS_NAME, 'jobs-search__results-list')
jobs = job_lists.find_elements(By.TAG_NAME, 'li') # return a list

In [5]:
len(jobs)

776

In [6]:
jobs[0]

<selenium.webdriver.remote.webelement.WebElement (session="655a9a101640fa01398aaed41b014f24", element="7ff7f69c-cc5b-41af-91c6-ab26c1afd245")>

In [7]:
start = time.time()

job_id= []
job_title = []
company_name = []
location = []
date = []
job_link = []

job_detail_list = list()

item = 1

for job in jobs:
    
    try:
        job_id0 = job.find_element(By.CSS_SELECTOR, 'div').get_attribute('data-entity-urn')
        job_id.append(job_id0)

        print(f'{item} {job_id0}')

        job_title0 = job.find_element(By.CSS_SELECTOR, 'h3').get_attribute('innerText')
        job_title.append(job_title0)

        company_name0 = job.find_element(By.CSS_SELECTOR, 'h4').get_attribute('innerText')
        company_name.append(company_name0)

        location0 = job.find_element(By.CSS_SELECTOR, '[class="job-search-card__location"]').get_attribute('innerText')
        location.append(location0)

        date0 = job.find_element(By.CSS_SELECTOR, 'div>div>time').get_attribute('datetime')
        date.append(date0)

        job_link0 = job.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')
        job_link.append(job_link0)

        job_detail = [''] * 6

        # clicking job to view job details
        job_click_path = f'/html/body/div[1]/div/main/section[2]/ul/li[{item}]/div/a'
        job_click = job.find_element(By.XPATH, job_click_path).click()
        time.sleep(5)

        # clicking "Show More"
        show_more_click_path = f'/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section/button[1]'
        show_more_click = job.find_element(By.XPATH, show_more_click_path).click()
        time.sleep(5)

        print(f'{item}: Clicked on Show More')

        jd_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/div/section/div'
        jd = job.find_element(By.XPATH, jd_path).get_attribute('innerText')
        job_detail[0] = jd

        # extracting additional info
        try:
            seniority_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[1]/span'
            seniority = job.find_element(By.XPATH, seniority_path).get_attribute('innerText')

            emp_type_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[2]/span'
            emp_type = job.find_element(By.XPATH, emp_type_path).get_attribute('innerText')

            job_func_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[3]/span'
            job_func = job.find_element(By.XPATH, job_func_path).get_attribute('innerText')

            industries_path = '/html/body/div[1]/div/section/div[2]/div/section[1]/div/ul/li[4]/span'
            industries = job.find_element(By.XPATH, industries_path).get_attribute('innerText')

            applicants_path = '/html/body/div[1]/div/section/div[2]/section/div/div[1]/div/h4/div[2]/figure/figcaption'
            applicants = job.find_element(By.XPATH, applicants_path).get_attribute('innerText')

        except Exception as e:
            pass

        job_detail[1] = seniority
        job_detail[2] = emp_type
        job_detail[3] = job_func
        job_detail[4] = industries
        job_detail[5] = applicants

        job_detail_list.append(job_detail)
    
    except Exception as e:
        print(e)
        pass
    
    item += 1

end = time.time()
print(f'Run time to scrape all job details: {(end - start)/60} minutes')

1 urn:li:jobPosting:3514649710
1: Clicked on Show More
2 urn:li:jobPosting:3515053487
2: Clicked on Show More
3 urn:li:jobPosting:3536050700
3: Clicked on Show More
4 urn:li:jobPosting:3514614927
4: Clicked on Show More
5 urn:li:jobPosting:3541907131
5: Clicked on Show More
6 urn:li:jobPosting:3535430240
6: Clicked on Show More
7 urn:li:jobPosting:3533760521
7: Clicked on Show More
8 urn:li:jobPosting:3525276147
8: Clicked on Show More
9 urn:li:jobPosting:3545530903
9: Clicked on Show More
10 urn:li:jobPosting:3531559934
10: Clicked on Show More
11 urn:li:jobPosting:3546056022
11: Clicked on Show More
12 urn:li:jobPosting:3532882171
12: Clicked on Show More
13 urn:li:jobPosting:3536751840
13: Clicked on Show More
14 urn:li:jobPosting:3544879891
14: Clicked on Show More
15 urn:li:jobPosting:3532881209
15: Clicked on Show More
16 urn:li:jobPosting:3543344138
16: Clicked on Show More
17 urn:li:jobPosting:3539126524
17: Clicked on Show More
18 urn:li:jobPosting:3534017483
18: Clicked on Sh

In [8]:
len(job_id)

776

In [9]:
len(job_detail_list)

770

In [10]:
job_data1 = pd.DataFrame({'ID': job_id,
                        'Date': date,
                        'Company': company_name,
                        'Title': job_title,
                        'Location': location,
                        'Link': job_link})

In [11]:
job_data2 = pd.DataFrame({'Description': [job_detail[0] for job_detail in job_detail_list],
                        'Level': [job_detail[1] for job_detail in job_detail_list],            
                        'Type': [job_detail[2] for job_detail in job_detail_list],
                        'Function': [job_detail[3] for job_detail in job_detail_list],
                        'Industry': [job_detail[4] for job_detail in job_detail_list],
                        'Applicants': [job_detail[5] for job_detail in job_detail_list]})

In [12]:
job_data1.tail()

,ID,Date,Company,Title,Location,Link
771,urn:li:jobPosting:3551306429,2023-03-31,ClearanceJobs,Affordability Analyst/Data Scientist with Secu...,"Laurel, MD",https://www.linkedin.com/jobs/view/affordabili...
772,urn:li:jobPosting:3551303404,2023-04-02,ClearanceJobs,Decision Scientist with Security Clearance,"Laurel, MD",https://www.linkedin.com/jobs/view/decision-sc...
773,urn:li:jobPosting:3551304434,2023-03-31,ClearanceJobs,Electro-Optic Data Scientist / System Engineer...,"Laurel, MD",https://www.linkedin.com/jobs/view/electro-opt...
774,urn:li:jobPosting:3553245730,2023-04-01,ClearanceJobs,Data Scientist Journeyman with Security Clearance,"Arlington, VA",https://www.linkedin.com/jobs/view/data-scient...
775,urn:li:jobPosting:3547519463,2023-03-28,ClearanceJobs,Sentinel (GBSD) Principal Data Scientist with ...,"Colorado Springs, CO",https://www.linkedin.com/jobs/view/sentinel-gb...


In [13]:
job_data2.tail()

,Description,Level,Type,Function,Industry,Applicants
765,Description Are you seeking an opportunity to ...,Entry level,Full-time,Engineering and Information Technology,Defense and Space Manufacturing,Be among the first 25 applicants
766,"Description Are you a ""systems thinker"" with a...",Entry level,Full-time,Engineering and Information Technology,Defense and Space Manufacturing,Be among the first 25 applicants
767,"Description Do the challenges of analyzing, te...",Entry level,Full-time,Engineering and Information Technology,Defense and Space Manufacturing,Be among the first 25 applicants
768,"Overview Since 2006, Intelligent Waves has del...",Entry level,Full-time,Engineering and Information Technology,Defense and Space Manufacturing,Be among the first 25 applicants
769,"At a Minimum, The Principal Data Scientist Sha...",Entry level,Full-time,Engineering and Information Technology,Defense and Space Manufacturing,Be among the first 25 applicants


In [14]:
# cleaning description column
job_data2['Description'] = job_data2['Description'].str.replace('\n', ' ')
job_data1.to_csv('LinkedIn_Data Scientist_0402_1.csv', index = False)
job_data2.to_csv('LinkedIn_Data Scientist_0402_2.csv', index = False)